# Phase 2 Review

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
import scipy as sp
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import math


pd.set_option('display.max_columns', 100)

### Check Your Data … Quickly
The first thing you want to do when you get a new dataset, is to quickly to verify the contents with the .head() method.

In [2]:
df = pd.read_csv('movie_metadata.csv')
print(df.shape)
df = df.dropna(subset = ['gross'])

(5043, 28)


In [3]:
df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,1873,Polly Walker,1.0,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5034,Color,Neill Dela Llana,35.0,80.0,0.0,0.0,Edgar Tancangco,0.0,70071.0,Thriller,Ian Gamazon,Cavite,589,0,Quynn Ton,0.0,jihad|mindanao|philippines|security guard|squa...,http://www.imdb.com/title/tt0428303/?ref_=fn_t...,35.0,English,Philippines,Not Rated,7000.0,2005.0,0.0,6.3,NaN,74
5035,Color,Robert Rodriguez,56.0,81.0,0.0,6.0,Peter Marquardt,121.0,2040920.0,Action|Crime|Drama|Romance|Thriller,Carlos Gallardo,El Mariachi,52055,147,Consuelo Gómez,0.0,assassin|death|guitar|gun|mariachi,http://www.imdb.com/title/tt0104815/?ref_=fn_t...,130.0,Spanish,USA,R,7000.0,1992.0,20.0,6.9,1.37,0
5037,Color,Edward Burns,14.0,95.0,0.0,133.0,Caitlin FitzGerald,296.0,4584.0,Comedy|Drama,Kerry Bishé,Newlyweds,1338,690,Daniella Pineda,1.0,written and directed by cast member,http://www.imdb.com/title/tt1880418/?ref_=fn_t...,14.0,English,USA,Not Rated,9000.0,2011.0,205.0,6.4,NaN,413
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,Alan Ruck,Shanghai Calling,1255,2386,Eliza Coupe,5.0,NaN,http://www.imdb.com/title/tt2070597/?ref_=fn_t...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


## Question 1

A Hollywood executive wants to know how much an R-rated movie released after 2000 will earn. The data above is a sample of some of the movies with that rating during that timeframe, as well as other movies. How would you go about answering her question? Talk through it theoretically and then do it in code.

What is the 95% confidence interval for a post-2000 R-rated movie's box office gross?

In [5]:
# begin by seeing our NA's
df.isna().sum()

color                          2
director_name                  3
num_critic_for_reviews         3
duration                       2
director_facebook_likes        3
actor_3_facebook_likes        13
actor_2_name                   6
actor_1_facebook_likes         4
gross                          0
genres                         0
actor_1_name                   4
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  13
facenumber_in_poster           7
plot_keywords                 40
movie_imdb_link                0
num_user_for_reviews           1
language                       3
country                        0
content_rating                64
budget                       268
title_year                     3
actor_2_facebook_likes         6
imdb_score                     0
aspect_ratio                 105
movie_facebook_likes           0
dtype: int64

In [6]:
df.dropna(subset = ['gross'], inplace=True)

In [7]:
# talk through your answer here
# first, we must clean the data so we have the exact information we would like
# drop our null_values
r = df[(df['title_year'] > 2000.0) & (df['content_rating'] == 'R')]

In [8]:
# do it in code here
def conf_interval(sample, q):
    '''
    Function input: sample 
    Function output: z-critical, Margin of error, Confidence interval
    '''
    # Number of observations with dataset
    sample_size = len(sample)
    
    # Finding the mean from our sample.
    x_hat = sample.mean()
   
    # t-critical value equation
    t = stats.t.ppf(q, len(sample)-1)  
    # for z-critical if we know pop stand dev stats.norm.ppf(q = 0.975)    
    # t-critical value for 95% confidence
    # Note that we use stats.t.ppf(q, len(sample)-1)  to get the desired t-critical value 
    # instead of (q = 0.950) because we want the positive or right tail t-critical value
    
    # Calculate the std from data
    stdev = sample.std()

    # Calculate the margin of error using formula given above
    moe = t * (stdev/math.sqrt(sample_size))

    # Calculate the confidence interval by applying margin of error to sample mean 
    # (mean - margin of error, mean+ margin of error)
    conf = (x_hat - moe, x_hat + moe)
    
    return t, moe, conf

In [9]:
# 95% confidence interval
t_critical, margin_of_error, confidence_interval = conf_interval(r['gross'],0.975)
print("t-critical value:")              
print(t_critical)         
print ("\nMargin of error")
print(margin_of_error)
print("\nConfidence interval:")
print(confidence_interval)

t-critical value:
1.9619313510908767

Margin of error
2206496.731820402

Confidence interval:
(25442351.706093505, 29855345.169734307)


In [ ]:
# We are 95% confident that the true population mean lies between 25593934.50 and 29703726

## Question 2a

Your ability to answer the first question has the executive excited and now she has many other questions about the types of movies being made and the differences in those movies budgets and gross amounts.

Read through the questions below and **determine what type of statistical test you should use** for each question and **write down the null and alternative hypothesis for those tests**.

- Is there a relationship between the number of Facebook likes for a cast and the box office gross of the movie?
- Do foreign films perform differently at the box office than non-foreign films?
- Of all movies created are 40% rated R?
- Is there a relationship between the language of a film and the content rating (G, PG, PG-13, R) of that film?
- Is there a relationship between the content rating of a film and its budget? 

# your answers here
Simple linear regression
Ho: b1 = 0
Ha: b1 != 0

Two sample - Difference between two means T-Test
Ho: xdomestic = xforeign
Ha: xdomestic != xforeign

one sample, proportions z-test
ho: p = .40
ha: p != 40

chi-square
Ho: distributions are equal
Ha: distributions are not equal

Anova
ho: xg = xpg = xpg13 = r
ha: not all means are equal

## Question 2b

Calculate the answer for the second question:

- Do foreign films perform differently at the box office than non-foreign films?

In [8]:
df = df.dropna(subset=['gross'])
df_usa = df[df.country == 'USA']['gross']
df_foreign = df[~(df.country=='USA')]['gross']

In [9]:
def diff_two_means(df1, df2, q):
    
    n1 = len(df1)
    n2 = len(df2)
    x_1 = df1.mean()
    x_2 = df2.mean()
    s1 = df1.std()
    s2 = df2.std()
    df1_2 = (n1+n2)-2
    t_crit = stats.t.ppf(q, df=df1_2)
     # critical value 
    numer = (x_1 - x_2) - 0
    denum = math.sqrt((s1**2/n1)+(s2**2/n2))
    delta_mu = numer/denum
    results = stats.ttest_ind(df1, df2, axis=0, nan_policy = 'omit', equal_var=False)
    p = 1-stats.t.cdf(delta_mu, df=df1_2)
    print(results)
    print(t_crit)
    print(delta_mu)
    print(p)
    if delta_mu > t_crit or delta_mu < - t_crit :
        print ("We reject null hypothesis. Results are statistically significant with t-value =", 
        round(results[0], 2), "critical t-value =", t_crit, "and p-value =", np.round((results[1]), 10))
    else:
        print ("We fail to reject the null hypothesis. Results are statistically insignifcant with t-value ==", 
        round(results[0], 2), ", critical t-value =", t_crit, "and p-value =", np.round((results[1]), 10))

diff_two_means(df_usa, df_foreign, .975)

Ttest_indResult(statistic=14.911721173786228, pvalue=5.784651659798822e-48)
1.960534816956006
14.911721173786235
0.0
We reject null hypothesis. Results are statistically significant with t-value = 14.91 critical t-value = 1.960534816956006 and p-value = 0.0


In [13]:
from scipy.stats import ttest_ind

In [14]:
ttest_ind(df_foreign, df_usa)

Ttest_indResult(statistic=-12.098302287742106, pvalue=3.863109466861356e-33)

## Question 3

Now that you have answered all of those questions, the executive wants you to create a model that predicts the money a movie will make if it is released next year in the US. She wants to use this to evaluate different scripts and then decide which one has the largest revenue potential. 

Below is a list of potential features you could use in the model. Create a new frame containing only those variables.

Would you use all of these features in the model?

Identify which features you might drop and why.

*Remember you want to be able to use this model to predict the box office gross of a film **before** anyone has seen it.*

- **budget**: The amount of money spent to make the movie
- **title_year**: The year the movie first came out in the box office
- **years_old**: How long has it been since the movie was released
- **genre**: Each movie is assigned one genre category like action, horror, comedy
- **avg_user_rating**: This rating is taken from Rotten tomatoes, and is the average rating given to the movie by the audience
- **actor_1_facebook_likes**: The number of likes that the most popular actor in the movie has
- **total_cast_facebook_likes**: The sum of likes for the three most popular actors in the movie
- **language**: the original spoken language of the film


In [15]:
df['genre'] = df.genres.apply(lambda x: x.split('|')[0])

In [17]:
model_data = df[['gross', 'budget', 'actor_1_facebook_likes', 'cast_total_facebook_likes', 'title_year', 'content_rating', 'genres']]

num of crit reviews and imdb score to be dropped 

# Variables to keep and take out:
**What to keep**
- budget we would keep as budget could be a huge factor in how much revenue the movie breaks.
- actor_1_facebook_likes will be good to have because the more popular the actor the more a movie should make
- language could be an indicator as if it is in english it reached a broader audience as moany peple speak english as a 1st 2nd or 3rd language
- this will not play affect as we have no idea what our avg_user_rating on rotten tomatoes is yet
- language of movie could be important as USA films could do better (english universal) than films in germany.

**What not to keep**
- title_year will have no affect on our outcome as we do not have a title year for our variable yet
- genre is categorized poorly. it may be a good idea to use it, but we would need to categorize each movie into a specific category and go from there (or two specific categories.
- avg_user_rating would be bad to use because there is no reviews as no one has seen the movie
- tot_cast_facebook_likes: we would have a problem with multicolinearity here between actor_1_facebook_likes

## Question 4a

Create the following variables:

- `years_old`: The number of years since the film was released.
- Dummy categories for each of the following ratings:
    - `G`
    - `PG`
    - `R`
    
Once you have those variables, create a summary output for the following OLS model:

`gross~cast_total_facebook_likes+budget+years_old+G+PG+R`

In [49]:
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,212204,1873,Polly Walker,1.0,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000


In [18]:
years_old = df['title_year'].max() - df['title_year']
model_data = pd.get_dummies(model_data, columns=['content_rating']).drop(columns='content_rating_PG-13')

In [20]:
# model_data.columns = ['gross', 'budget', 'actor_1_facebook_likes', 'cast_total_facebook_likes', 'title_year', 'years_old', 'G', 'PG', 'R']

ValueError: Length mismatch: Expected axis has 18 elements, new values have 9 elements

In [85]:
mlr_model = ols(formula = 'gross~cast_total_facebook_likes+budget+years_old+title_year+durat', data=df).fit()
mlr_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  gross   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     128.8
Date:                Thu, 21 Jan 2021   Prob (F-statistic):          2.02e-103
Time:                        11:38:16   Log-Likelihood:                -75535.
No. Observations:                3890   AIC:                         1.511e+05
Df Residuals:                    3885   BIC:                         1.511e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                  -180.6398     52.917     -3.414      0.001    -284.387     -76.893
cast_total_facebook_likes   753.9301     56.588     13.323      0.000     642.986     864.874
budget                        0.0245      0.005      5.155      0.000       0.015       0.034
years_old                 -3.487e+05   1.07e+05     -3.260      0.001   -5.58e+05   -1.39e+05
title_year                -1.551e+04   2604.708     -5.954      0.000   -2.06e+04   -1.04e+04
duration                    7.01e+05   4.73e+04     14.809      0.000    6.08e+05    7.94e+05
==============================================================================
Omnibus:                     2269.877   Durbin-Watson:                   1.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30833.291
Skew:                           2.524   Prob(JB):                         0.00
Kurtosis:                      15.835   Cond. No.                     7.74e+23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.34e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [72]:
df[['duration', 'gross']].corr()

,duration,gross
duration,1.000000,0.250298
gross,0.250298,1.000000


In [ ]:
# your answer here

## Question 4b

Below is the summary output you should have gotten above. Identify any key takeaways from it.
- How ‘good’ is this model?
- Which features help to explain the variance in the target variable? 
    - Which do not? 


<img src="ols_summary.png" style="withd:300px;">

In [ ]:
# your answer here


## Question 5

**Bayes Theorem**

An advertising executive is studying television viewing habits of married men and women during prime time hours. Based on the past viewing records he has determined that during prime time wives are watching television 60% of the time. It has also been determined that when the wife is watching television, 40% of the time the husband is also watching. When the wife is not watching the television, 30% of the time the husband is watching the television. Find the probability that if the husband is watching the television, the wife is also watching the television.

In [84]:
# your answer here
a = .6
not_a = .4
b_a = .4
b_not_a =.3

b = (b_a*a) + (b_not_a*not_a)
print(b)
b_a = (b_a * a) / b

b_a

0.36


0.6666666666666666

## Question 6

Explain what a Type I error is and how it relates to the significance level when doing a statistical test. 

# your answer here
the probability of rejecting the null hypothesis when the null hypothesis is in fact true. or a false positive.